# Double pendulum with `python` and `sympy`

We want to solve the equations of motion for the double pendulum. It is a classics in Theoretical physics which cannot be solved generally without numerics.

<img src="figs/double_pendulum.png" height=100 />

## Lagrangian Mechanics

The (simplified) idea of Lagrangian mechanics is the that kinetic energy and potential energy of a system can be expressed in terms of generalized coordinates and velocities

$$T = T(q,\dot{q}) \hspace{20mm} V=V(q,\dot{q})$$

where $q$ could be $x, y, z$ of a point particle, or some angle $\theta$ (pendulum), or whatever makes the problem easiest to solve. The Lagragian $L=T-V$ together with the following equation gives the equations of motion $q(t)$

$$\frac{dL}{dq} - \frac{d}{dt}\frac{dL}{d\dot{q}} = 0 $$

for all the different $q$'s of the system. The equation above gives a second order differential equation for all the different $q(t)$s.

In the following, we will use `sympy` to build up the Lagragian of the double pendulum and solve the resulting differential equations numerically with `odeint`.

In [ ]:
import sympy as sp
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt
from matplotlib import animation

Define relevant `sympy`-quantities: 

In [ ]:
L1, L2, t, m1, m2, g = sp.symbols(r'L_1 L_2 t m_1 m_2 g')
theta1, theta2 = sp.symbols(r'\theta_1 \theta_2', cls=sp.Function)
theta1 = theta1(t)
theta2 = theta2(t)

In [ ]:
theta1_d = sp.diff(theta1, t)
theta2_d = sp.diff(theta2, t)
theta1_dd = sp.diff(theta1_d, t)
theta2_dd = sp.diff(theta2_d, t)

Define $x,y$-location of pendulum. This allows us to easily obtain necessary potential and kinetic energies of the system:

In [ ]:
x1 = L1 * sp.sin(theta1)
y1 = -L1 * sp.cos(theta1)
x2 = x1 + L2 * sp.sin(theta2)
y2 = y1 - L2 * sp.cos(theta2)

Get kinetic and potential energies:

In [ ]:
T1 = sp.Rational(1, 2) * m1 * (sp.diff(x1, t)**2 + sp.diff(y1, t)**2)
V1 = m1 * g * y1
T2 = sp.Rational(1, 2) * m2 * (sp.diff(x2, t)**2 + sp.diff(y2, t)**2)
V2 = m2 * g * y2
T = T1 + T2
V = V1 + V2

In [ ]:
T.simplify()

Get Lagragian:

In [ ]:
L = T - V
L.simplify()

Get Lagrange's equations

$$\frac{\partial L}{\partial \theta_1} - \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_1}} = 0$$
$$\frac{\partial L}{\partial \theta_2} - \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_2}} = 0$$

In [ ]:
LE1 = sp.diff(L, theta1) - sp.diff(sp.diff(L, theta1_d), t)
LE2 = sp.diff(L, theta2) - sp.diff(sp.diff(L, theta2_d), t)

In [ ]:
LE1.simplify()

In [ ]:
LE2.simplify()

From `LE1` and `LE2`, we want to extract a system of two second order differential equations that we can solve.

The following `solve` command gives us equations

$$
\frac{d^2\theta_1}{dt^2} = \dots
$$
and
$$
\frac{d^2\theta_2}{dt^2} = \dots
$$


**Notes:** (1) Lagranges equation ensure that the second derivatives $d^2 q / dt^2$ (in our case $d^2 \theta_1 / dt^2$ and $d^2 \theta_2 / dt^2$) *always* appear as linear factors!; (2) The following `sp.solve` assumes that `LE1` and `LE2` are both equal to zero.

In [ ]:
sol = sp.solve([LE1, LE2], [theta1_dd, theta2_dd])

In [ ]:
sol[theta1_dd].simplify()

Now we have 

* $\frac{d^2 \theta_1}{dt^2} = ...$
* $\frac{d^2 \theta_2}{dt^2} = ...$


**Recap from the `scipy`-lecture:**

These are two second order ODEs! In python we can only solve systems of first order ODEs. Any system of second order ODEs can be converted as follows:

1. Define $u = d\theta_1/dt$ and $v=d\theta_2/dt$
2. Then $du/dt = d^2\theta_1/dt^2$ and $dv/dt = d^2\theta_2/dt^2$

Now we get a system of 4 first order ODEs (as opposed to 2 second order ones)

* $d\theta_1/dt = u$
* $d\theta_2/dt = v$
* $d u/dt = ...$
* $d v/dt = ...$

We need to convert the **symbolic** expressions above to numerical functions so we can use them in a numerical python solver. For this we use `smp.lambdify`

In [ ]:
theta1_dd_n = sp.lambdify([theta1, theta2, 
                           theta1_d, theta2_d, 
                           L1, L2, m1, m2, g], sol[theta1_dd])
theta2_dd_n = sp.lambdify([theta1, theta2, 
                           theta1_d, theta2_d, 
                           L1, L2, m1, m2, g], sol[theta2_dd])

In [ ]:
theta1_dd_n(0.0, np.pi/4, 0.0, 0.0, 1, 1, 1, 1, 9.81)

Now define $\vec{S} = (\theta_1, \theta_2, u, v)$. If we're going to use an ODE solver in python, we need to write a function that takes in $\vec{S}$ and $t$ and returns $d\vec{S}/dt$. In other words, we need to define $\frac{d\vec{S}(\vec{S}, t)}{dt}$

In [ ]:
def dSdt(S, t, L1, L2, m1, m2, g):
    theta1, theta2, u, v = S
    
    dtheta1dt = u
    dtheta2dt = v
    dudt = theta1_dd_n(theta1, theta2, u, v, L1, L2, m1, m2,g)
    dvdt = theta2_dd_n(theta1, theta2, u, v, L1, L2, m1, m2,g)
    
    return [dtheta1dt, dtheta2dt, dudt, dvdt]

Fix physical constants:

In [ ]:
L1, L2, m1, m2, g = 4., 4., 0.5, 0.5, 9.81

In [ ]:
t = np.linspace(0.0, 20, 1001)
S0 = [3.0 * np.pi / 4., 0.0, 2.0, -1.0]
#S0 = [np.pi / 4., 0.0, 0.0, 0.0]
sol_dgl = si.odeint(dSdt, S0, t=t, args=(L1, L2, m1, m2, g))

In [ ]:
plt.plot(t, sol_dgl[:,0])

Finally, create an animation of our double pendulum:

In [ ]:
def get_x1y1x2y2(theta1, theta2):
    return (L1 * np.sin(theta1),
            - L1 * np.cos(theta1),
            L1 * np.sin(theta1) + L2 * np.sin(theta2),
            -L1 * np.cos(theta1) - L2 * np.cos(theta2)
    )

In [ ]:
x1, y1, x2, y2 = get_x1y1x2y2(sol_dgl[:,0], sol_dgl[:,1])

In [ ]:
plt.plot(x2)

In [ ]:
def animate(i, ln1):
    ln1.set_data([0, x1[i], x2[i]], [0, y1[i], y2[i]])
    
    return ln3,

fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.grid()
ln1, = plt.plot([], [], lw=3, markersize=15, marker='o', 
                color='black', markerfacecolor='lightblue')

ax.set_ylim(-10, 10)
ax.set_xlim(-10,10)
ani = animation.FuncAnimation(fig, animate, fargs=(ln1,),
                              frames=1000, interval=50)
ani.save('pendulum.gif',writer='pillow',fps=50)